This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [14]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#15.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [15]:
BoSSSshell.WorkflowMgm.Init("CNS_AcousticWave1D_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'CNS_AcousticWave1D_ConvStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D_ConvStudy'.


{ Session Count = 60; Grid Count = 60; Path = \\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D_ConvStudy }

In [16]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-05-03 10:54:50Z

## Choose which runs to plot

In [17]:
var ses=database.Sessions.Where(s => (s.Name.Contains("sP1.5") &&s.Name.Contains("ampneg0")&&s.Name.Contains("amppos1E-05")&& !s.Name.Contains("241"))); // the one plotted in the publication
ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses


#0: AcousticWave	AW_p3_xCells121_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:59:46 AM	9aa361cb...
#1: AcousticWave	AW_p3_xCells61_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:59:32 AM	5d9caf6a...
#2: AcousticWave	AW_p3_xCells31_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:59:14 AM	43c9286c...
#3: AcousticWave	AW_p2_xCells121_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:58:27 AM	e6fe2b8a...
#4: AcousticWave	AW_p3_xCells16_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:58:58 AM	75dcede5...
#5: AcousticWave	AW_p2_xCells61_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:58:09 AM	efb9a79d...
#6: AcousticWave	AW_p2_xCells31_yCells3_sP1.5_pST10_wP-0.8_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus_RK4	5/3/2024 10:57:54 AM	db6c132e...
#7: AcousticWave	AW_p2_xCells16_yCells3

# Helper Functions to obtain the error metrics

In [19]:
(double[] tVals, double[]yVals) GetPlotAlongXRay(double tMin,double tMax, double x, int nRef,string field,ISessionInfo si, double t0){
//double shockPosition=0.5;
var p1= new double[] {x,tMin};
var p2=new double[] {x,tMax};

//timesteps to compare
double[] tVals= GenericBlas.Linspace(tMin,tMax,nRef);
double[] yVals=new double[tVals.Length];
for(int it=0;it<tVals.Length;it++){
    var t=tVals[it];
    //closest timestep to time t
    var tiEnd=si.Timesteps.Where(ti =>ti.PhysicalTime >t-1e-2).OrderBy(ti => Math.Abs(ti.PhysicalTime - t))
            .FirstOrDefault();
    tVals[it]=tiEnd.PhysicalTime;
    var pEnd =tiEnd.GetField(field);
    var yValpEnd = pEnd.ProbeAt(new double[]{x,0.0015}); // evaluate field at
    // substract the t0 Base to only obtain the pertubations
    var ti=si.Timesteps.Where(ti =>Math.Abs(ti.PhysicalTime - t0)<1e-1).OrderBy(ti => Math.Abs(ti.PhysicalTime - t0))
            .FirstOrDefault();
    var p0=ti.GetField(field);
    var yValp0 = p0.ProbeAt(new double[]{x,0.0015}); 
    yVals[it]=yValpEnd-yValp0;
}
return(tVals,yVals);
}
(double[] xVals, double[]yVals, double tPhys) GetPlotAlongTRay(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si, double t0){
//double shockPosition=0.5;
var p1= new double[] {xMin,0.0015};
var p2=new double[] {xMax,0.0015};

//closest timestep to time t
var tiEnd=si.Timesteps.Where(ti =>ti.PhysicalTime >t-1e-2).OrderBy(ti => Math.Abs(ti.PhysicalTime - t))
            .FirstOrDefault();

var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);

// substract the t0 Base to only obtain the pertuabtions
var ti=si.Timesteps.Where(ti =>Math.Abs(ti.PhysicalTime - t0)<1e-1).OrderBy(ti => Math.Abs(ti.PhysicalTime - t0))
            .FirstOrDefault();
var tOField=ti.GetField(field);
double[] yVals_tOField=tOField.EvaluateAlongLine(p1,p2,nRef);
yVals.AccV(-1.0,yVals_tOField);

double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals,tiEnd.PhysicalTime);
}
List<double> ComputeAmpErrorsAlongX(double tMin,double tMax, double xMin, double xMax, int xRef, int tRef,string field,ISessionInfo si, bool isMax,double exact_amp_fac, double t0){
    var xVals= GenericBlas.Linspace(xMin,xMax,xRef);
    var maxminVals = new List<double>();
    foreach(var x in xVals ){
        (double[] tVals, double[] yVals)=GetPlotAlongXRay(tMin,tMax,x,tRef,field,si,t0);
        if(isMax){
            maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Max())/exact_amp_fac));
        }else{
            maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Min())/exact_amp_fac));
        }
    }
    return maxminVals;
}
List<double> ComputeAmpErrors(double tMin,double tMax, double xMin, double xMax, int xRef, int tRef,string field,ISessionInfo si, bool isMax,double exact_amp_fac, double t0){
    var tVals= GenericBlas.Linspace(tMin,tMax,tRef);
    var xVals=GenericBlas.Linspace(xMin,xMax,xRef);
    var allY= MultidimensionalArray.Create(xVals.Length,tVals.Length);
    var maxminVals = new List<double>();
    for(int iCol=0;iCol<tVals.Length;iCol++){
        (double[] xVals_dummy, double[] yVals, double tPhys) =GetPlotAlongTRay(xMin,xMax,tVals[iCol],xRef,field,si,t0);
        allY.SetColumn(iCol,yVals);
    }
    allY.SaveToTextFile("CNSFastAcc.txt");
    for(int iRow=0;iRow<xVals.Length;iRow++){
        var yVals=allY.GetRow(iRow);
        if(isMax){
            maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Max())/exact_amp_fac));
        }else{
            maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Min())/exact_amp_fac));
        }
    }
    return maxminVals;
}

var eps=0.000001;
double xMin=1.8 +eps;double xMax=2.7-eps; int xRef=40; 
double tMin =0 + eps;int tRef=100;

In [22]:
var ampfactors=ComputeAmpErrors(tMin,tMax, xMin, xMax, 10,100,"p",si,true,exact_amp_fac,9.95); ampfactors

[ 0.012747617197489028, 0.01183564673460224, 0.013940706688975336, 0.019044215871185155, 0.015396550060239626, 0.012191290495611315, 0.011974095179230168, 0.014820197585040511, 0.019457188133115026, 0.1830931537240339 ]

Number of cells

In [23]:
double[] Cells = ses.Where(si => si.Name.Contains("p0")).Select(si => (double) si.GetGrids().Pick(0).NumberOfCells).Distinct().ToArray();
Cells

[ 363, 183, 93, 48 ]

cell lengths in $x$-direction

In [24]:
var Hs= ses.Select(si => si.Timesteps.Last().Fields.Pick(0).Basis.GridDat.iGeomCells.GetCellVolume(0)/0.003).Distinct().ToArray();
Hs

[ 0.08264462809917356, 0.16393442622950818, 0.3225806451612903, 0.625 ]

helper arrays

In [25]:
int[] degs = new int[] {0,1,2,3};
//degs = new int[] {0};
var errors_array= MultidimensionalArray.Create(degs.Count(),ses.Count());


Function used to calculate the EOCs

In [26]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

Function used to compute the ideal convergence plot

In [27]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

compute the exact analytical pressure amplification/reduction factor for both cases investigated

In [28]:
// Given variables
var MBaseL = 1.5; // Base Mach number on the left
var gamma = 1.4; //Specific heat ratio

// Calculate right Mach number (Mach_R)
var MBaseR  = Math.Sqrt((1 + ((gamma-1)/2) * MBaseL*MBaseL) / (gamma * MBaseL*MBaseL - (gamma-1)/2));

// Upstream fast acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrL = ((1+MBaseL)*(1+MBaseL)) / (1+2*MBaseR+1/MBaseL/MBaseL) * (1 - ((gamma-1)/(gamma+1)) * (1-(1/MBaseL))* (1-(1/MBaseL)));

// Downstream slow acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrMiR = -(1- 2*MBaseR +1 / MBaseL/MBaseL) / (1+ 2*MBaseR +1 / MBaseL/ MBaseL);
(delta_pPrR_over_delta_pPrL,delta_pPrR_over_delta_pPrMiR)

Item1,2.154925878735875
Item2,-0.01484811194806477


helper function to obtain different plot formats

In [29]:
using BoSSS.Solution.Tecplot;
using System.IO;
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [30]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(Cells.Length,degs.Length +1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(Cells.Length,degs.Length+1);
for(int iSess =0;iSess < ses.Count();iSess++){
    var si = ses.Pick(iSess);
    //extract the relevant field and timestep Index
    var timesteps = si.Timesteps;
    var tEnd= timesteps.Last();
    var rho=tEnd.GetField("rho");
    int p= rho.Basis.Degree;
    int iCell= Cells.FirstIndexWhere(c => c==rho.Basis.GridDat.iGeomCells.Count);
    Console.Write($"P={p},cells={Cells[iCell]/3}");
    //extract AmpError
    if(si.Name.Contains("sP0.5")){
        double exact_amp_fac=delta_pPrR_over_delta_pPrMiR*1e-05; double tMin=15.8 +eps;double tMax=tMin+4.0-eps;
        if(tEnd.PhysicalTime>tMax){
            List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p",si,false,exact_amp_fac,t0:9.95);
            //errors_array[p,iSess]=ampErrors.Average();
            errorsAndCells[iCell,p+1]=(ampErrors.Max());
            Console.WriteLine($" sp0.5, h: {Hs[iCell]}, error: {errorsAndCells[iCell,p+1]}");
        }
    }else{
        double exact_amp_fac=delta_pPrR_over_delta_pPrL*1e-05; double tMin=10.2 +eps;double tMax=tMin+1.0-eps;
        if(tEnd.PhysicalTime>tMax){
            List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p",si,true,exact_amp_fac,t0:9.95);
            //errors_array[p,iSess]=ampErrors.Average();
            errorsAndCells[iCell,p+1]=(ampErrors.Max());
            Console.WriteLine($" sp1.5, h: {Hs[iCell]}, error: {errorsAndCells[iCell,p+1]}");
        }
    }

}  


P=3,cells=121 sp1.5, h: 0.08264462809917356, error: 0.16651939209219527
P=3,cells=61 sp1.5, h: 0.16393442622950818, error: 0.1868559366460145
P=3,cells=31 sp1.5, h: 0.3225806451612903, error: 0.28964620544861996
P=2,cells=121 sp1.5, h: 0.08264462809917356, error: 0.15917403485692266
P=3,cells=16 sp1.5, h: 0.625, error: 0.30659718748965065
P=2,cells=61 sp1.5, h: 0.16393442622950818, error: 0.38003478106605293
P=2,cells=31 sp1.5, h: 0.3225806451612903, error: 0.16109851904500566
P=2,cells=16 sp1.5, h: 0.625, error: 0.32744419260083185
P=1,cells=121 sp1.5, h: 0.08264462809917356, error: 0.20637015751072993
P=1,cells=61 sp1.5, h: 0.16393442622950818, error: 0.24218775868372597
P=1,cells=31 sp1.5, h: 0.3225806451612903, error: 0.3767651558106156
P=1,cells=16 sp1.5, h: 0.625, error: 0.6381081523822829
P=0,cells=121P=0,cells=61P=0,cells=31P=0,cells=16

In [31]:
errorsAndCells.ToMsrMatrix().ToStringDense()

             0	             0	 2.063702E-001	 1.591740E-001	 1.665194E-001
             0	             0	 2.421878E-001	 3.800348E-001	 1.868559E-001
             0	             0	 3.767652E-001	 1.610985E-001	 2.896462E-001
             0	             0	 6.381082E-001	 3.274442E-001	 3.065972E-001


In [32]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();

MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(Cells.Length,degs.Length+1);
errorsAndCells.SetColumn(0,Hs.ToList());
errorsAndCellsIdeal.SetColumn(0,Hs.ToList());
int count=0;
foreach(int p in degs){
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errorsAndCells.GetColumn(p+1).ToList(),Hs.ToList(),p));
    EOCs.Add( ApproxSlope(errorsAndCells.GetColumn(p+1).ToList(),Hs.ToList()));
    plot.AddDataGroup($"P={p}",Hs,errorsAndCells.GetColumn(p+1), GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;
    //plot.AddDataGroup($"ideal {p+1}",Hs,GetIdealConvPlot(errorsAndCells.GetColumn(p+1).ToList(),Hs.ToList(),p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("CNS_ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("CNS_ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Hs";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Hs 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3

In [33]:
si =ses.Pick(0);
IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)si.Timesteps.Last()).GetInternal();
double exact_amp_fac=-0.014848*1e-05; double tMin=0.0 +eps;double tMax=4.0-eps;
List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,false,exact_amp_fac);
ampErrors.Average()

Error: (4,25): error CS1501: No overload for method 'ComputeAmpErrors' takes 10 arguments

In [34]:
(double[] tVals, double[] yVals)=GetPlotAlongXRay(tMin,tMax,2.7-eps,tRef,"p_per",lts);
yVals.Min();
Math.Abs((exact_amp_fac-yVals.Min())/exact_amp_fac)

Error: (1,82): error CS0103: The name 'lts' does not exist in the current context

In [35]:
EOCs

[ NaN, 0.5610482633134605, 0.3583693789320386, 0.30059387435908175 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [36]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlux0"));
sessions

In [37]:
var si = sessions.Pick(0);
si

Error: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'index')
   at System.Linq.ThrowHelper.ThrowArgumentOutOfRangeException(ExceptionArgument argument)
   at System.Linq.Enumerable.ElementAt[TSource](IEnumerable`1 source, Int32 index)
   at BoSSS.Foundation.IO.IEnumerableExtensions.Pick[T](IEnumerable`1 entities, Int32 index) in C:\experimental\public\src\L4-application\BoSSSpad\IEnumerableExtensions.cs:line 131
   at Submission#38.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

plot it

In [38]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=5;
var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
foreach(var si in ses){
    foreach(var timestep in si.Timesteps){
        if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
                IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
                var SF = lts.GetShadowFields();
                var texplot = new Tecplot(timestep.Fields.Pick(0).GridDat, 2);
                texplot.PlotFields( si.Name + "_" + lts.TimeStepNumber,0.0,timestep.Fields);
        }
    }
}

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#39.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)